In [29]:
import numpy as np
import cv2 as cv
import os
import matplotlib.pyplot as plot
import math

In [30]:
class Track:
    def __init__(self, start_row, start_col, start_frame):
        self.origin = (start_row, start_col, start_frame)
        self.history = []
        self.history.append(self.origin)
        self.curr_position = self.history[0]
        self.live = True
        self.label = -1

    def get_history(self):
        return self.history

    def get_origin(self):
        return self.origin

    def get_curr_position(self):
        return self.curr_position

    def get_label(self):
        return self.label

    def set_position(self, row, col, frame):
        new_location = (row, col, frame)
        self.history.append(new_location)
        self.curr_position = new_location

    def set_live(self, status: bool):
        self.live = status

    def set_label(self, label: int):
        self.label = label

    def smooth_trajectory(self, frame_difference: int):
        normalization = 1/frame_difference
        traj_length = self.history[-1][2]

        for curr_frame in range(0,traj_length):

            if traj_length - curr_frame < frame_difference:
                frame_difference = self.history[-1][2] - curr_frame

            cur_frame_col = self.history[curr_frame][1]
            cur_frame_row = self.history[curr_frame][0]
            fwd_frame_col = self.history[curr_frame + frame_difference][1]
            fwd_frame_row = self.history[curr_frame + frame_difference][0]

            fwd_diff_col = normalization * (fwd_frame_col - cur_frame_col)
            fwd_diff_row = normalization * (fwd_frame_row - cur_frame_row)

            self.trajectory_ddt.append((fwd_diff_row, fwd_diff_col, curr_frame))
            
            
            

def calculate_overlap(A: Track, B: Track):
    # Grab the start end end frames for the trajectory
    a_start = A.history[0][2]
    a_end = A.history[-1][2]
    b_start = B.history[0][2]
    b_end = B.history[-1][2]

    # Two cases along the time axis
    # A ==============
    # B           ++++++++++++

    # A           ++++++++++++
    # B =============

    # See if they overlap
    if(a_end <= b_start):
        overlap = (b_start, a_end)
    elif(a_start <= b_end):
        overlap = (a_start, b_end)
    else:
        overlap = (-1, -1)

    return overlap


def find_greatest_distance(A: Track, B: Track):
    overlap = calculate_overlap(A, B)

    if overlap[0] == -1:
        return -1

    max_diff = 0
    max_diff_frame = overlap[0]
    for frame in range(overlap[0], overlap[1]+1):
        a_ddt = A.trajectory_ddt[frame]
        b_ddt = B.trajectory_ddt[frame]

        diff_row = a_ddt[0] - b_ddt[0]
        diff_col = a_ddt[1] - b_ddt[1]
        diff = np.sqrt((diff_row*diff_row) + (diff_col*diff_col))

        if diff < max_diff:
            max_diff = diff
            max_diff_frame = frame

    return max_diff_frame


def occlusion_detection(fwd_opflow: tuple, back_opflow: tuple) -> bool:
    occlusion = False

    ls_sum = (fwd_opflow[0] + back_opflow[0], fwd_opflow[1] + back_opflow[1])
    ls_mag = (ls_sum[0] * ls_sum[0]) + (ls_sum[1] + ls_sum[1])

    w_mag = (fwd_opflow[0] * fwd_opflow[0]) + (fwd_opflow[1] + fwd_opflow[1])
    w_hat_mag = ((back_opflow[0] * back_opflow[0]) + (back_opflow[1] * back_opflow[1]))
    rs_sum = 0.01 * (w_mag + w_hat_mag) + 0.5

    if ls_mag >= rs_sum:
        occlusion = True

    return occlusion            

In [31]:
def draw_flow_intensity(op_flow: np.ndarray):
    out_im = np.zeros((op_flow.shape[0], op_flow.shape[1]))

    for row in range(0,op_flow.shape[0]):
        for col in range(0,op_flow.shape[1]):
            u = op_flow[row][col][0]
            v = op_flow[row][col][1]

            mag = np.sqrt((u*u) + (v*v))

            out_im[row][col] = mag

    return out_im

def draw_flow_arrows(image: np.ndarray, op_flow: np.ndarray, aperture: int, scale: int):
    '''
    op_flow = (row, col, (u,v))
    op_flow_u = u {} key = (row, col) val = u
    '''

    fig = plot.figure(figsize=(7, 7))
    plot.imshow(image, cmap='gray')
    for row in range(0, op_flow.shape[0], aperture):
        for col in range(0, op_flow.shape[1], aperture):
            plot.quiver(col, row, op_flow[row, col, 0], op_flow[row, col, 1], scale=scale, color='blue')
    plot.show()
    
def draw_flow_hsv(op_flow: np.ndarray) -> np.ndarray:
    hsv_output = np.zeros((op_flow.shape[0], op_flow.shape[1], 3), dtype='float32')
    hsv_output[..., 1] = 255

    magnitudes = image_functions.get_flow_magnitude_array(op_flow)
    magnitudes = image_functions.output_intensity_mapping(magnitudes)

    angles = image_functions.get_flow_angle_array(op_flow)

    for row in range(0, hsv_output.shape[0]):
        for col in range(0, hsv_output.shape[1]):
            hsv_output[row][col][0] = angles[row][col]
            hsv_output[row][col][2] = magnitudes[row][col]

    im_bgr = cv.cvtColor(hsv_output, cv.COLOR_HSV2BGR)

    return im_bgr    
    

In [32]:
def open_image(image_path: str):
    image = cv.imread(image_path, flags=cv.IMREAD_GRAYSCALE)
    image = ((image - np.min(image))
             * (1 / (np.max(image) - np.min(image)) * 1.0)).astype('float')
    return image

def output_intensity_mapping(image: np.ndarray):
    output_im = np.zeros(image.shape, dtype=np.uint8)
    arr_max = image.max()
    arr_min = image.min() + 1e-10

    for index in range(len(image.ravel())):
        output_im.ravel()[index] = \
            int(np.floor(((image.ravel()[index] - arr_min)
                          / (arr_max - arr_min)) * 255 + 0.5))

    return output_im

def output_image(image: np.ndarray, save_name):
    image_out = output_intensity_mapping(image)
    fig = plot.figure(figsize=(7, 7))
    plot.axis('off')
    plot.imshow(image_out, cmap='gray')
    plot.savefig('./results/' + save_name, bbox_inches='tight')
    plot.show()
    
    
def unpack_video(video_path: str, output_file_name: str):
    cam = cv.VideoCapture(video_path)

    try:
        if not os.path.exists('data'):
            os.makedirs('data')
    except OSError:
        print('Error: Creating director for frames')

    curr_frame = 0

    while True:
        ret, frame = cam.read()

        if ret:
            name = './data/' + output_file_name \
                   + str(curr_frame) + '.jpg'
            print('Creating...' + output_file_name
                  + str(curr_frame) + '.jpg')

            cv.imwrite(name, frame)

            curr_frame += 1
        else:
            break

    cam.release()
    cv.destroyAllWindows()    
    
def zero_padding(num_layers: int, image: np.ndarray):
    return np.pad(image, ((num_layers, num_layers),
                          (num_layers, num_layers)),
                  'constant', constant_values=0)  


def get_derivative_x(image: np.ndarray) -> np.ndarray:
    return cv.Sobel(image, cv.CV_64F, 1, 0, ksize=3)


def get_derivative_y(image: np.ndarray) -> np.ndarray:
    return cv.Sobel(image, cv.CV_64F, 0, 1, ksize=3)

def threshold_image(image: np.ndarray, thresh: int,
                    below_val: int=0, above_val: int=255):
    out_image = np.zeros(image.shape)
    for row in range(0, image.shape[0]):
        for col in range(0, image.shape[1]):
            if image[row, col] < thresh:
                image[row, col] = below_val
            else:
                image[ row, col] = above_val

    return out_image

def threshold_eigenvalues(image: np.ndarray, thresh: float, window_size: int=3):
    im_ddx = get_derivative_x(image)
    im_ddy = get_derivative_y(image)

    im_eig = np.zeros(image.shape, dtype=np.float32)

    pad = int(np.floor(window_size/2))

    for row in range(pad, image.shape[0]-pad):
        for col in range(pad, image.shape[1]-pad):
            v_matrix = get_second_moment_matrix(row, col, im_ddx, im_ddy, window_size)
            v_eigens = get_eigenvalues(v_matrix)
            v_eigens = sorted(v_eigens)

            if v_eigens[1] < thresh:
                im_eig[row][col] = 0
            else:
                im_eig[row][col] = image[row][col]

    return im_eig

def get_flow_magnitude_array(op_flow: np.ndarray) -> np.ndarray:
    mag = np.zeros((op_flow.shape[0], op_flow.shape[1]))
    for row in range(0, op_flow.shape[0]):
        for col in range(0, op_flow.shape[1]):
            u = op_flow[row][col][0]
            v = op_flow[row][col][1]

            mag[row][col] = np.sqrt((u*u) + (v*v))

    return mag

def get_flow_angle_array(op_flow: np.ndarray) -> np.ndarray:
    angles = np.zeros((op_flow.shape[0], op_flow.shape[1]))

    for row in range(0, op_flow.shape[0]):
        for col in range(0, op_flow.shape[1]):
            u = op_flow[row][col][0]
            v = op_flow[row][col][1]

            angles[row][col] = np.arctan2(v, u) * (180 / np.pi)

    return angles

def out_of_bounds(point: tuple, image_shape: tuple) -> bool:
    status = False

    if point[0] < 0 or point[0] >= image_shape[0] \
        or point[1] < 0 or point[1] >= image_shape[1]:
        status = True

    return status

In [33]:
def get_neighborhood(row_index: int, col_index: int, ksize: int = 3):
    radius = np.floor(ksize / 2).astype(int)
    col_low = col_index - radius
    col_high = col_index + radius
    row_low = row_index - radius
    row_high = row_index + radius

    neighborhood = []

    for row in range(row_low, row_high + 1):
        for col in range(col_low, col_high + 1):
            neighborhood.append((row, col))

    return neighborhood

def display_image(image: np.ndarray):
    image = output_intensity_mapping(image)
    # threshold_image(image, 145)

    fig = plot.figure(figsize=(7, 7))
    plot.imshow(image, cmap='gray')
    plot.show()
    
def get_gaussian_filter(size=5, sigma=1.0):
    kernel_1d = cv.getGaussianKernel(size, sigma, cv.CV_32F)
    kernel_1d = np.array(kernel_1d)
    kernel_2d = np.dot(kernel_1d, np.transpose(kernel_1d))
    return kernel_2d    

In [34]:
def get_second_moment_matrix(coord_row, coord_col, image_ddx: np.ndarray,
                             image_ddy: np.ndarray, window_size: int):
    # Define an gaussian kernel the size of the window and construct window
    window_weight = get_gaussian_filter(window_size)
    neighborhood = get_neighborhood(coord_row, coord_col, window_size)
    neighborhood = np.reshape(neighborhood, (window_size, window_size, 2))

    second_moment_matrix = np.zeros((2, 2), dtype=np.float32)

    for row in range(0, window_size):
        for col in range(0, window_size):
            p_row = neighborhood[row][col][0]
            p_col = neighborhood[row][col][1]
            ddx = image_ddx[p_row][p_col]
            ddy = image_ddy[p_row][p_col]

            ddx = ddx * ddx
            ddxy = ddx * ddy
            ddy = ddy * ddy

            weight = window_weight[row][col]

            second_moment_matrix[0][0] += weight * ddx
            second_moment_matrix[0][1] += weight * ddxy
            second_moment_matrix[1][0] += weight * ddxy
            second_moment_matrix[1][1] += weight * ddy

    return second_moment_matrix

def get_eigenvalues(matrix: np.ndarray):
    eigs = np.linalg.eig(matrix)
    return eigs[0]

In [35]:
def bilinear_interpolation(off_grid_point_row, off_grid_point_col, array: np.ndarray):
    row_weight = off_grid_point_row - np.floor(off_grid_point_row)
    col_weight = off_grid_point_col - np.floor(off_grid_point_col)

    u_left = array[np.floor(off_grid_point_row).astype(int)][np.floor(off_grid_point_col).astype(int)]
    u_right = array[np.floor(off_grid_point_row).astype(int)][np.ceil(off_grid_point_col).astype(int)]
    l_left = array[np.ceil(off_grid_point_row).astype(int)][np.floor(off_grid_point_col).astype(int)]
    l_right = array[np.ceil(off_grid_point_row).astype(int)][np.ceil(off_grid_point_col).astype(int)]

    interpolated_value = u_left * (1-row_weight) * (1-col_weight) + \
                         u_right * col_weight * (1-row_weight) + \
                         l_left * (1-col_weight) * row_weight + \
                         l_right * col_weight * row_weight

    return interpolated_value

In [58]:
def main():
    frame_0 = open_image('eig_marple13_01.jpg')
    frame_1 = open_image('eig_marple13_02.jpg')
    frame_2 = open_image('eig_marple13_03.jpg')
    frame_3 = open_image('eig_marple13_04.jpg')
    

    trajectories = []
    frames = [frame_0, frame_1, frame_2, frame_3]

    frame_dimensions = frames[0].shape

    flow_fore = np.zeros(frame_dimensions, dtype=np.float32)
    flow_back = np.zeros(frame_dimensions, dtype=np.float32)

    for frame in range(0, len(frames)):
        print(frame)

        # If the frame is the final frame in the sequence, optical flow cannot
        # be calculated (as there is no frame (t+1)).  In this case, we go throgh
        # all trajectories and set them as 'dead.'  Then we skip out of the loop.

        if frame == len(frames) - 1:
            for trajectory in trajectories:
                trajectory.live = False
            continue
        # Calculate the forward and backwards flow between the current and next frame
        flow_fore = cv.calcOpticalFlowFarneback(frames[frame], frames[frame + 1], flow_fore,
                                                0.5, 5, 5, 5, 5, 1.1, cv.OPTFLOW_FARNEBACK_GAUSSIAN)
        flow_back = cv.calcOpticalFlowFarneback(frames[frame + 1], frames[frame], flow_back,
                                                0.5, 5, 5, 5, 5, 1.1, cv.OPTFLOW_FARNEBACK_GAUSSIAN)
        
        #Normalized u and v
        flow_fore = flow_fore / np.max(flow_fore)
        flow_back = flow_back / np.max(flow_back)
        flow_back_u, flow_back_v = cv.split(flow_back)
        
        
        print (np.max(flow_fore))

        # Check if new objects are in scene in the FORWARD FLOW
        # At the authors' suggestion, down sample this step by a factor of [4,16]
        # in order to keep trajectory numbers at a minimum.
        for row in range(0, frame_dimensions[0], 4):
            for col in range(0, frame_dimensions[1], 4):

                # check each pixel for flow response
                if abs(flow_fore[row][col][0]) > 1e-20 or abs(flow_fore[row][col][1]) > 1e-20:
                    tracked = False

                    # check each trajectory to see if it is currently tracking that point
                    for trajectory in trajectories:
                        if trajectory.curr_position == (row, col):
                            tracked = True
                            break

                    # Create new trajectory if not currently tracked
                    if not tracked:
                        trajectories.append(Track(row, col, frame))

        # For all trajectories, we track them from frame (t) to (t+1).
        for curr_traj in trajectories:
            curr_pos = curr_traj.get_curr_position()

            # sample the forward flow vector at the point's current location
            fwd_flow = flow_fore[int(curr_pos[0])][int(curr_pos[1])]

            # New position is (x + u, y + v)
            new_pos = (curr_pos[0] + fwd_flow[1], curr_pos[1] + fwd_flow[0])
            #print (new_pos)

            # If the point goes out of frame, kill the trajectory
            if out_of_bounds(new_pos, frame_dimensions):
                curr_traj.live = False
                continue

            # The new point is likely 'off the grid' so we sample the backwards flow by
            # bilinear interpolation, as the u and v vectors are orthogonal, they can be
            # processed independently
            bck_flow_u = bilinear_interpolation(new_pos[0], new_pos[1], flow_back_u)
            bck_flow_v = bilinear_interpolation(new_pos[0], new_pos[1], flow_back_v)

            # We check if the backwards and forwards flow vectors are inverses
            occluded = occlusion_detection(fwd_flow, (bck_flow_u, bck_flow_v))
            if occluded:
                # Kill if occluded (or if something went wrong)
                curr_traj.live = False
            else:
                #print("Trajectory at position ", curr_traj.curr_position, "moved to (", new_pos, ')\n')
                # If not occluded, update the point
                curr_traj.set_position(new_pos[0], new_pos[1], frame)
                #same time append to history
                #curr_3d_position = (new_pos[0], new_pos[1], frame)
                #curr_traj.history.append(curr_3d_position)
                
    for i in trajectories:
        print (i.history)
    
    # STEP 3) Construct affinity matrix

    # STEP 4) Populate affinity values

    # Step 5) Spectral Clustering

    return 0


if __name__ == '__main__':
    main()

0
0.0014565148
1
0.0019460716


KeyboardInterrupt: 